In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import json

In [2]:
IEX_CLOUD_API_TOKEN='Tpk_059b97af715d417d9f49f50b51b1c448'


In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data




{'avgTotalVolume': 88538133,
 'calculationPrice': 'close',
 'change': 0.7,
 'changePercent': 0.00483,
 'close': 154.17,
 'closeSource': 'fcoiialf',
 'closeTime': 1723025909033,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 151.389,
 'delayedPriceTime': 1726982540012,
 'extendedChange': -0.28,
 'extendedChangePercent': -0.0019,
 'extendedPrice': 152.3,
 'extendedPriceTime': 1661095868205,
 'high': 154.17,
 'highSource': 'a d eic1uy nte5leidpmre',
 'highTime': 1708395425277,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 151.45,
 'iexCloseTime': 1666300177108,
 'iexLastUpdated': 1683701267982,
 'iexMarketPercent': 0.015405211412711908,
 'iexOpen': 151.94,
 'iexOpenTime': 1689804490866,
 'iexRealtimePrice': 152.03,
 'iexRealtimeSize': 103,
 'iexVolume': 993782,
 'lastTradeTime': 1717629255037,
 'latestPrice': 147.41,
 'latestSource': 'Close',
 'latestTime': 'July 8, 2022',
 'latestUpdate': 1730358445134,
 'latestVolume': 65051512

In [5]:

data['latestPrice']
data['marketCap']


2414086412059

In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:

final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

/tmp/ipykernel_725706/3995666295.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,147.41,2414086412059,N/A


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

/tmp/ipykernel_725706/184330454.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/184330454.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/184330454.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/184330454.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/184330454.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [9]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,126.03,38096979786,N/A
1,AAL,14.39,9446274513,N/A
2,AAP,185.3,11334657708,N/A
3,AAPL,154.11,2465175937986,N/A
4,ABBV,160.05,271036601721,N/A
...,...,...,...,...
496,YUM,121.01,33762953589,N/A
497,ZBH,105.37,22557285502,N/A
498,ZBRA,309.82,16002528402,N/A
499,ZION,51.8,7766362694,N/A


In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

/tmp/ipykernel_725706/2255717043.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/2255717043.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/2255717043.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/2255717043.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/tmp/ipykernel_725706/2255717043.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,125.96,37408454356,N/A
1,AAL,14.18,9345040893,N/A
2,AAP,184,11133671255,N/A
3,AAPL,149.8,2442324924803,N/A
4,ABBV,153.83,272402920626,N/A
...,...,...,...,...
496,YUM,118.27,33964996222,N/A
497,ZBH,105.65,22566916389,N/A
498,ZBRA,302.33,16107761327,N/A
499,ZION,52.19,7761738831,N/A


In [12]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,125.96,37408454356,N/A
1,AAL,14.18,9345040893,N/A
2,AAP,184,11133671255,N/A
3,AAPL,149.8,2442324924803,N/A
4,ABBV,153.83,272402920626,N/A
...,...,...,...,...
496,YUM,118.27,33964996222,N/A
497,ZBH,105.65,22566916389,N/A
498,ZBRA,302.33,16107761327,N/A
499,ZION,52.19,7761738831,N/A


In [13]:
portfolio_size=input('Enter the value of your portfolio: ')
try:
    val=float(portfolio_size)
except ValueError:
    print('That`s not number! \nPlease try again')
    portfolio_size=input('Enter the value of your portfolio: ')
    val=float(portfolio_size)
    

In [14]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
     final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
    # number_of_apple_shares=position_size/500
# print(math.floor(number_of_apple_shares))

final_dataframe



,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,125.96,37408454356,1
1,AAL,14.18,9345040893,14
2,AAP,184,11133671255,1
3,AAPL,149.8,2442324924803,1
4,ABBV,153.83,272402920626,1
...,...,...,...,...
496,YUM,118.27,33964996222,1
497,ZBH,105.65,22566916389,1
498,ZBRA,302.33,16107761327,0
499,ZION,52.19,7761738831,3


In [35]:
writer=pd.ExcelWriter('recommended trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,"Recommended Trades",index=False)


In [36]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'0.00$',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [37]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)

# writer.save()
writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Share to Buy',integer_format)

0

In [34]:
column_formats={
'A':['Ticker',string_format],
'B':['Stock',dollar_format],
'C':['Market Capitalization',dollar_format],
'D':['Number Of Shares to Buy',integer_format
]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

In [ ]:
writer.save()